In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import cv2
import numpy as np
import pandas as pd

In [28]:
def load_train_images_from_folder(folder_path):
    
    image_filenames = os.listdir(folder_path)
    images = []
    i = 0
    for image_filename in image_filenames:
        
        if i < 150 :
            image = cv2.imread(os.path.join(folder_path, image_filename))
            images.append(image)
            i = i+1

    return np.array(images)
def load_test_images_from_folder(folder_path):
    
    image_filenames = os.listdir(folder_path)
    images = []
    i = 0
    for image_filename in image_filenames:
        if i < 45 :
            image = cv2.imread(os.path.join(folder_path, image_filename))
            images.append(image)
            i = i+1

    return np.array(images)

In [29]:
test_fake_images = load_train_images_from_folder("C://Users//pajag//OneDrive//Desktop//Dataset//Test//Fake")

In [30]:
test_real_images = load_test_images_from_folder("C://Users//pajag//OneDrive//Desktop//Dataset//Test//Real")

In [31]:
train_fake_images = load_train_images_from_folder("C://Users//pajag//OneDrive//Desktop//Dataset//Train//Fake")

In [32]:
train_real_images = load_train_images_from_folder("C://Users//pajag//OneDrive//Desktop//Dataset//Train//Real")

In [33]:
IMAGE_SIZE = (224, 224)
def preprocess_images(array):
    resized = []
    for img in array:
        img = cv2.resize(img, IMAGE_SIZE)
        img = img / 255.0
        resized.append(img)
    return resized

In [34]:
train_fake_images = preprocess_images(train_fake_images)
train_real_images = preprocess_images(train_real_images)
test_real_images  = preprocess_images(test_real_images)
test_fake_images  = preprocess_images(test_fake_images)

In [35]:
fake_train_df = pd.DataFrame({'image': train_fake_images, 'label': '0'})

In [36]:
real_test_df = pd.DataFrame({'image': test_real_images, 'label': '1'})

In [37]:
fake_test_df = pd.DataFrame({'image': test_fake_images, 'label': '0'})

In [38]:
real_train_df = pd.DataFrame({'image': train_real_images, 'label': '1'})

In [39]:
test_df = pd.concat([real_test_df,fake_test_df])

In [40]:
test_df.head()

,image,label
0,"[[[0.26666666666666666, 0.13333333333333333, 0...",1
1,"[[[0.5882352941176471, 0.6352941176470588, 0.8...",1
2,"[[[0.15294117647058825, 0.1568627450980392, 0....",1
3,"[[[0.43137254901960786, 0.4392156862745098, 0....",1
4,"[[[0.5450980392156862, 0.6078431372549019, 0.5...",1


In [41]:
train_df = pd.concat([real_train_df,fake_train_df])

In [42]:
train_df.head()

,image,label
0,"[[[0.26666666666666666, 0.13333333333333333, 0...",1
1,"[[[0.5882352941176471, 0.6352941176470588, 0.8...",1
2,"[[[0.15294117647058825, 0.1568627450980392, 0....",1
3,"[[[0.43137254901960786, 0.4392156862745098, 0....",1
4,"[[[0.5450980392156862, 0.6078431372549019, 0.5...",1


In [43]:
train_df = train_df.iloc[np.random.permutation(len(train_df))]

In [44]:
test_df = test_df.iloc[np.random.permutation(len(test_df))]

In [45]:
train_df.head()

,image,label
129,"[[[0.23137254901960785, 0.19215686274509805, 0...",1
112,"[[[0.9529411764705882, 0.9686274509803922, 0.9...",0
146,"[[[0.8862745098039215, 0.8941176470588236, 0.8...",0
116,"[[[0.2784313725490196, 0.2901960784313726, 0.3...",1
45,"[[[0.43529411764705883, 0.4666666666666667, 0....",1


In [46]:
test_df.iloc[1]

image    [[[0.9490196078431372, 0.9529411764705882, 0.9...
label                                                    1
Name: 11, dtype: object

In [47]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [48]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [49]:
x_batch = train_df['image']
y_batch = train_df['label']

In [50]:
x_batch = [tf.convert_to_tensor(img) for img in x_batch]
x_batch = tf.stack(x_batch)
y_batch = tf.convert_to_tensor(y_batch)
y_batch = tf.strings.to_number(y_batch, out_type=tf.float32)

In [51]:
model.fit(x_batch,y_batch,batch_size=128,epochs=5)

Epoch 1/5
3/3 [==============================] - 20s 5s/step - loss: 1.4448 - accuracy: 0.4767
Epoch 2/5
3/3 [==============================] - 14s 4s/step - loss: 0.8819 - accuracy: 0.5200
Epoch 3/5
3/3 [==============================] - 14s 4s/step - loss: 0.6552 - accuracy: 0.5333
Epoch 4/5
3/3 [==============================] - 13s 4s/step - loss: 0.5964 - accuracy: 0.7733
Epoch 5/5
3/3 [==============================] - 13s 4s/step - loss: 0.4690 - accuracy: 0.8700


In [56]:
def evaluate_model(data):
    test_images = [tf.convert_to_tensor(image) for image in data['image'].values]
    test_images = tf.stack(test_images)

    test_labels = tf.convert_to_tensor(data['label'].values)
    test_labels = tf.strings.to_number(test_labels, out_type=tf.float32)

    loss, accuracy = model.evaluate(test_images, test_labels)
    return accuracy

In [57]:
evaluate_model(test_df)

7/7 [==============================] - 3s 393ms/step - loss: 0.4709 - accuracy: 0.8359


0.8358974456787109

In [59]:
def save_model(model, filename):
    model.save(filename)

In [60]:
save_model(model, "deepfake_detection_img_model.h5")

C:\Users\pajag\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
